In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import geopandas as gpd
from shapely.geometry import Point

In [200]:
# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [201]:
# 데이터 불러오기
dong_gdf = gpd.read_file('BND_ADM_DONG_PG_dong/seoul_dong_2018.shp', encoding="UTF-8")
gu_gdf = gpd.read_file('LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11_202405.shp', encoding="CP949")
school = pd.read_csv('school_2023_secondhalf.csv')

year = 232
student_column = f'counts_{year}'
output_student_column = f'student_count_{year}'
output_school_column = f'school_count_{year}'

In [202]:
# 학생 수 컬럼을 int로 변환 (변환 실패 시 NaN으로 처리)
school[student_column] = pd.to_numeric(school[student_column], errors='coerce')
# NaN 값 제거
school = school.dropna(subset=[student_column])

In [203]:
# school.info()

In [204]:
# 학교 데이터를 GeoDataFrame으로 변환
school['geometry'] = school.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)
school_gdf = gpd.GeoDataFrame(school, geometry='geometry', crs="EPSG:4326")

In [205]:
# 좌표계 통일 (EPSG:5181 - Korea Central Belt 2010)
dong_gdf = dong_gdf.to_crs("EPSG:5181")
gu_gdf = gu_gdf.to_crs("EPSG:5181")
school_gdf = school_gdf.to_crs("EPSG:5181")

In [206]:
# 행정동과 학교 데이터 Spatial Join (학교가 속한 행정동 찾기)
joined_dong = gpd.sjoin(school_gdf, dong_gdf, how="inner", predicate="within")

In [207]:
# 행정동별로 학교 수 계산
school_counts = joined_dong.groupby('adm_nm').size().reset_index(name=output_school_column)
# 행정동별로 학생 수 총합 계산
student_counts = (joined_dong.groupby('adm_nm')[student_column]
                  .sum()
                  .reset_index(name=output_student_column)
)

In [208]:
# 병합: 행정동별 학교 수와 학생 수 결합
dong_school_data = pd.merge(school_counts, student_counts, on='adm_nm', how='inner')

In [209]:
# 동 데이터 확인
print("총 동 개수:", len(dong_school_data))
print(dong_school_data)

총 동 개수: 390
             adm_nm  school_count_232  student_count_232
0    서울특별시 강남구 개포1동                 3              795.0
1    서울특별시 강남구 개포2동                 8             6881.0
2    서울특별시 강남구 개포4동                 4             1950.0
3    서울특별시 강남구 논현1동                 1              235.0
4    서울특별시 강남구 논현2동                 2              776.0
..              ...               ...                ...
385   서울특별시 중랑구 묵2동                 2              827.0
386  서울특별시 중랑구 상봉1동                 4             3120.0
387  서울특별시 중랑구 신내1동                 2             1516.0
388  서울특별시 중랑구 신내2동                 3             1566.0
389  서울특별시 중랑구 중화1동                 6             2467.0

[390 rows x 3 columns]


In [210]:
# CSV 저장
output_file = f'dong_school_{year}.csv'
dong_school_data.to_csv(output_file, index=False, columns=['adm_nm', output_school_column, output_student_column])

print(f"CSV 저장 완료: {output_file}")

CSV 저장 완료: dong_school_232.csv
